<h1 style="text-align:center"> Basis set optimization with chemtools tutorial</h1>

In [1]:
from chemtools.basisset import BasisSet
from chemtools.basisopt import BSOptimizer
from chemtools.molecule import Molecule
from chemtools.molpro import Molpro

Define the program that will be used to perform the energy calculations, in this particular case it will be [Molpro](https://www.molpro.net/).

In [2]:
mp = Molpro(executable="/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin/molpro",
            runopts=["-s", "-n", "1", "-d", "."])

# Optimization of even tempered parameters at the Hartree-Fock level for Be atom

In the first example we will optimize the *s* exponents for the HF calculations.

define the system for which the optimization will be performed

In [3]:
be = Molecule('Be', atoms=[('Be',)])

template for the input file to be used in sigle point calculations

In [4]:
templ =  '''***,be
memory,100,m

%geometry

%basis

%core

gthresh,energy=1.0e-9
{rhf; wf,4,1,0}

'''

In [5]:
bso = BSOptimizer(method='hf', objective='total energy', template=templ, code=mp, mol=be,
                  fsopt={'Be' : [('s', 'et', 8, (0.2, 4.0)),],}, verbose=False)

In [6]:
bso.run()

Script name : /usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py
Workdir     : /usr/local/lib/python2.7/dist-packages/IPython/kernel
Start time  : 2015-07-03 00:11:13.710915

=============================STARTING OPTIMIZATION==============================

======================================CODE======================================
<Molpro(
	name=Molpro,
	molpropath=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin,
	executable=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin/molpro,
	scratch=/home/lmentel/scratch,
	runopts=['-s', '-n', '1', '-d', '.'],
)>
======================================MOL=======================================
Name: Be         Charge: 0          Multiplicty: 1          Electrons: 4         
Atoms:
Element    Nuclear Charge	       x              y              z       
Be                   4.00	        0.00000        0.00000        0.00000
=====================================OPTALG=====================================
{'jaco

In [7]:
bso.result

  status: 0
    nfev: 70
 success: True
     fun: -14.566522375031999
       x: array([ 0.07026115,  3.55530678])
 message: 'Optimization terminated successfully.'
     nit: 36

# Optimization of diffuse functions for cc-pvdz for Be 

Since Be atom doesn't form a stable negative ion diffuse functions for Be are optimized for BeH<sup>-</sup>  

In [8]:
beh = Molecule(name="Be", atoms=[('Be',), ('H', (0.0, 0.0, 2.724985))], sym="cnv 2", charge=-1, multiplicity=1)

In [9]:
bsstr = '''basis={
s,Be,2.940000E+03,4.412000E+02,1.005000E+02,2.843000E+01,9.169000E+00,3.196000E+00,1.159000E+00,1.811000E-01,5.890000E-02;
c,1.9,6.800000E-04,5.236000E-03,2.660600E-02,9.999300E-02,2.697020E-01,4.514690E-01,2.950740E-01,1.258700E-02,-3.756000E-03;
c,1.9,-1.230000E-04,-9.660000E-04,-4.831000E-03,-1.931400E-02,-5.328000E-02,-1.207230E-01,-1.334350E-01,5.307670E-01,5.801170E-01;
c,9.9,1.000000E+00;

p,Be,3.619000E+00,7.110000E-01,1.951000E-01,6.018000E-02;
c,1.4,2.911100E-02,1.693650E-01,5.134580E-01,4.793380E-01;
c,4.4,1.000000E+00;

d,Be,2.354000E-01;
c,1.1,1.000000E+00;

s, H , 13.0100000, 1.9620000, 0.4446000, 0.1220000, 0.0297400
c, 1.3, 0.0196850, 0.1379770, 0.4781480
c, 4.4, 1
c, 5.5, 1
p, H , 0.7270000, 0.1410000
c, 1.1, 1
c, 2.2, 1
}
'''
diffusetmp =  '''***,be
memory,100,m

%geometry

%basis

%core

gthresh,energy=1.0e-9
{rhf; wf,6,1,0}
cisd

'''

In [10]:
bsd = BasisSet.from_str(bsstr, fmt='molpro', name='cc-pvdz')

In [11]:
difffs = {'Be' : [('s', 'exp', 1, (0.02,)), ('p', 'exp', 1, (0.01,)), ('d', 'exp', 1, (0.07,))]}

In [12]:
bso = BSOptimizer(method='cisd', objective='total energy', template=diffusetmp, code=mp, mol=beh,
                  fsopt=difffs, staticbs=bsd, core=[1,0,0,0,0,0,0,0], verbose=False)

In [13]:
bso.run()

Script name : /usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py
Workdir     : /usr/local/lib/python2.7/dist-packages/IPython/kernel
Start time  : 2015-07-03 00:11:43.699471

=============================STARTING OPTIMIZATION==============================

======================================CODE======================================
<Molpro(
	name=Molpro,
	molpropath=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin,
	executable=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin/molpro,
	scratch=/home/lmentel/scratch,
	runopts=['-s', '-n', '1', '-d', '.'],
)>
======================================MOL=======================================
Name: Be         Charge: -1         Multiplicty: 1          Electrons: 6         
Atoms:
Element    Nuclear Charge	       x              y              z       
Be                   4.00	        0.00000        0.00000        0.00000
H                    1.00	        0.00000        0.00000        2.72499
===============

In [14]:
bso.result

  status: 0
    nfev: 106
 success: True
     fun: -15.158346988063
       x: array([ 0.02079341,  0.18930133,  0.07711282])
 message: 'Optimization terminated successfully.'
     nit: 61

# Optimization of tight functions for cc-pvdz Be

In [15]:
bsd = BasisSet.from_str(bsstr, fmt='molpro', name='cc-pvdz')
pvdzbe = {k:v for k, v in bsd.items() if k=='Be'}

In [16]:
tightfs = {'Be' : [('s', 'exp', 1, (1.8,)), ('p', 'exp', 1, (4.2,))]}

In [17]:
tighttmp = '''***,be-core
memory,100,m                            !allocate 500 MW dynamic memory

%geometry

%basis

%core

{rhf; wf,4,1,0}
cisd
'''

In [18]:
bso = BSOptimizer(method='cisd', objective='core energy', template=tighttmp, code=mp, mol=be,
                  fsopt=tightfs, staticbs=pvdzbe, core=[[1,0,0,0,0,0,0,0],[0,0,0,0,0,0,0,0]], verbose=False)

In [19]:
bso.run()

Script name : /usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py
Workdir     : /usr/local/lib/python2.7/dist-packages/IPython/kernel
Start time  : 2015-07-03 00:12:34.024122

=============================STARTING OPTIMIZATION==============================

======================================CODE======================================
<Molpro(
	name=Molpro,
	molpropath=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin,
	executable=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin/molpro,
	scratch=/home/lmentel/scratch,
	runopts=['-s', '-n', '1', '-d', '.'],
)>
======================================MOL=======================================
Name: Be         Charge: 0          Multiplicty: 1          Electrons: 4         
Atoms:
Element    Nuclear Charge	       x              y              z       
Be                   4.00	        0.00000        0.00000        0.00000
=====================================OPTALG=====================================
{'jaco

In [20]:
bso.result

  status: 0
    nfev: 68
 success: True
     fun: -0.031657195983001429
       x: array([ 1.85981969,  6.16310027])
 message: 'Optimization terminated successfully.'
     nit: 34

# Optimization of mid-bond function exponents

In [21]:
be2X = Molecule(name="Be2", atoms=[('Be', (0.0, 0.0, -1.5)),
                                  ('H', (0.0, 0.0,  0.0), True),
                                  ('Be', (0.0, 0.0,  1.5))], sym="dnh 2", charge=0, multiplicity=1)

In [22]:
mbfs = {'H' : [('s', 'et', 4, (0.05, 2.0)), ('p', 'et', 4, (0.04, 2.0))]}

In [23]:
mbtmp = '''***,h2o test
memory,100,m                            !allocate 500 MW dynamic memory

%geometry

%basis

dummy, H

%core

{rhf; wf,8,1,0}
cisd

'''

In [24]:
bso = BSOptimizer(method='cisd', objective='total energy', template=mbtmp, code=mp, mol=be2X,
                  fsopt=mbfs, staticbs=pvdzbe, core=[2,0,0,0,0,0,0,0], verbose=False)

In [25]:
bso.run()

Script name : /usr/local/lib/python2.7/dist-packages/IPython/kernel/__main__.py
Workdir     : /usr/local/lib/python2.7/dist-packages/IPython/kernel
Start time  : 2015-07-03 00:13:43.444477

=============================STARTING OPTIMIZATION==============================

======================================CODE======================================
<Molpro(
	name=Molpro,
	molpropath=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin,
	executable=/home/lmentel/Programs/molprop_2012_1_Linux_x86_64_i8/bin/molpro,
	scratch=/home/lmentel/scratch,
	runopts=['-s', '-n', '1', '-d', '.'],
)>
======================================MOL=======================================
Name: Be2        Charge: 0          Multiplicty: 1          Electrons: 8         
Atoms:
Element    Nuclear Charge	       x              y              z       
Be                   4.00	        0.00000        0.00000       -1.50000
H                    0.00	        0.00000        0.00000        0.00000
Be             

In [26]:
bso.result

  status: 2
    nfev: 168
 success: False
     fun: -29.164707647050999
       x: array([ 0.05114879,  1.70176733,  0.05504829,  1.9460615 ])
 message: 'Maximum number of iterations has been exceeded.'
     nit: 100

In [27]:
%version_information chemtools

Software versions
Python 2.7.9 64bit [GCC 4.9.2]
IPython 3.2.0
OS Linux 3.16.0 4 amd64 x86_64 with debian 8.1
chemtools 0.5.0
Fri Jul 03 00:15:00 2015 CEST